In [111]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tabulate import tabulate

pd.options.display.float_format = '{:,}'.format
table_style = [{'selector': 'tr:nth-of-type(odd)',
  'props': [('background', '#eee')]}, 
 {'selector': 'tr:nth-of-type(even)',
  'props': [('background', 'white')]},
 {'selector': 'th',
  'props': [('background', '#606060'), 
            ('color', 'white'),
            ('font-family', 'verdana'),
            ("font-size", "150%")]},
 {'selector': 'td',
  'props': [('font-family', 'verdana'), 
            ("font-size", "150%")]},
]

# Isolated Stats

In [112]:
df = pd.read_csv('seq.csv')
df = df.drop(columns=['gpusim_version', 'jobId', 'config'])
df = df.rename(columns={'pair_str': 'benchmark'})
print(df.columns)

Index(['benchmark', 'grid_x', 'grid_y', 'grid_z', 'block_x', 'block_y',
       'block_z', 'ctas/SM', 'runtime', 'instructions', 'avg_mem_lat',
       'avg_core_to_l2', 'avg_l2_to_core', 'avg_mrq_latency',
       'stall_core_ldst', 'stall_icnt_to_l2', 'l2_rshr_entry_fail',
       'l2_rshr_merge_fail', 'stall_l2_to_icnt', 'l1D_miss_rate',
       'l2_miss_rate'],
      dtype='object')


### Basic Characteristics

In [113]:
df['grid'] = df['grid_x'] * df['grid_y'] * df['grid_z']
df['block'] = df['block_x'] * df['block_y'] * df['block_z']
# df['runtime'] = df['runtime'].astype(float)
df['achieved ctas/SM'] = pd.DataFrame([basic['grid'] / 80, df['ctas/SM']]).min() 


basic = df[['benchmark', 'grid', 'block', 'ctas/SM', 'achieved ctas/SM', 'runtime', 'instructions']]


# style the table
basic.style.set_table_styles(table_style).hide_index()

benchmark,grid,block,ctas/SM,achieved ctas/SM,runtime,instructions
cut_sgemm-0,128,256,2,1.6,91668,228392960
parb_stencil-0,1024,128,16,12.8,334891,644994210
parb_sgemm-0,528,128,11,6.6,654458,2800545792
parb_lbm-0,18000,120,12,12,929616,427778603
parb_spmv-0,1147,128,16,14.3375,73031,52720795
parb_cutcp-0,121,128,16,1.5125,525052,7055002292
cut_wmma-0,128,64,4,1.6,62335,36372480


### Memory Latency

In [114]:
df['time(l2/dram)'] = df['avg_mem_lat'] - df['avg_core_to_l2'] - df['avg_l2_to_core']

l2_hit_lat = 120
dram_lat = 100
df['predict(l2/dram)'] = l2_hit_lat + dram_lat * df['l2_miss_rate']

mem = df[['benchmark', 'avg_mem_lat', 'avg_core_to_l2', 
          'avg_l2_to_core', 'avg_mrq_latency', 'time(l2/dram)', 'predict(l2/dram)']]


# style the table
mem.style.set_table_styles(table_style).hide_index()

benchmark,avg_mem_lat,avg_core_to_l2,avg_l2_to_core,avg_mrq_latency,time(l2/dram),predict(l2/dram)
cut_sgemm-0,315,150,22,6,143,174.66
parb_stencil-0,1277,694,20,170,563,196.51
parb_sgemm-0,223,32,11,15,180,134.45
parb_lbm-0,21834,21115,58,187,661,219.96
parb_spmv-0,796,241,15,76,540,205.71
parb_cutcp-0,154,22,7,6,125,124.53
cut_wmma-0,836,704,8,8,124,132.75


### Stalls

In [142]:
stall = df[['stall_core_ldst', 'stall_icnt_to_l2', 'l2_rshr_entry_fail',
       'l2_rshr_merge_fail', 'stall_l2_to_icnt']].div(df[['runtime']].iloc[:, 0], axis=0)

stall['benchmark'] = df['benchmark']

stall.style.set_table_styles(table_style).hide_index()


###### CHECK if the values are correct!!!!!!!!!!!!

stall_core_ldst,stall_icnt_to_l2,l2_rshr_entry_fail,l2_rshr_merge_fail,stall_l2_to_icnt,benchmark
5.82374,0.319566,0,0.173147,0,cut_sgemm-0
11.8458,21.4548,0,0,0,parb_stencil-0
11.2983,0.00609359,0,3.14369,0,parb_sgemm-0
9.21082,19.6344,0.0299489,0,0,parb_lbm-0
36.9625,13.8805,0.000301242,0.000944804,0,parb_spmv-0
3.72643,0.0122274,0,0,0,parb_cutcp-0
13.6165,0.197963,0,0,0,cut_wmma-0
